In [1]:
import os 
import time
import copy
from tqdm import tqdm

import numpy as np 
import matplotlib.pyplot as plt
import scipy
from scipy.linalg import ishermitian
import pandas as pd

from joblib import Parallel, delayed
from numba import jit, njit, prange
import functools as ft

############## Local imports ##############
from hamiltonian_generation import make_H4_sparse

########### Define macros ############
np.random.seed(0)

K=7
J=4
Q=4
N=2*K
N_DIM = 2**K

############ Load test coefficients ############
js_fpath = os.path.join("Excel", 'test_coefficients_K{}_J{}_q{}.csv'.format(K,J,Q))
js_test = np.fromfile(js_fpath, sep = ',').reshape(tuple([N for i in range(Q)]))
js_test[0,0,0]

array([ 0.32995524,  0.07484697,  0.18306698,  0.41914542,  0.34931534,
       -0.18279387,  0.17770825, -0.02831044, -0.01930646,  0.07679995,
        0.02694247,  0.272013  ,  0.14234747,  0.02275857])

## Generate Hamiltonian

In [2]:
H4_func, psi, psi_pairs, psi_quads = make_H4_sparse(K, J, precompute_pairs = True, precompute_quads = True) # <-- This is the function being tested, by having its output compared to that of Mathematica
H4_test = H4_func(js_test)

In [ ]:
iv_test = np.linalg.eigvalsh(H4_test)
print(f"Eigenvalues mostly real: {np.allclose(iv_test, np.real(iv_test))}")
plt.hist(iv, bins=6)
print(np.sort(iv.real))

Boom, exactly the same eigenvalues as the Mathematica method